## 02 Data cleaning

> This notebook cleans the data and prepare it for the data exploration and the tableau visualizations 

>The aim is to deal with nans. As we have seen there is a lot. Also to delete data we don´t need and prepare some outputs to be
>used in Tableau
>For EDA check next notebook and Tableau dashboards

In [1]:
import pandas as pd
import warnings

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
pd.set_option('display.max_rows', None) # display all rows 
warnings.filterwarnings('ignore') # ignore warnings

In [2]:
# 🔧 Basic functions
def open_data(data): # returns shape, data types & shows a small sample
    print(f"Data shape is {data.info()}.\n")
    return data.sample(5)

def clean_columns(data): # clean columns names
    data.columns = [column.lower().replace(' ', '_') for column in data.columns]
    return data.sample(0)

def explore_data(data): # sum & returns duplicates, NaN & empty spaces added a table with percentage missing
    duplicate_rows = data.duplicated().sum()
    nan_values = data.isna().sum()
    empty_spaces = data.eq(' ').sum()
    exploration = pd.DataFrame({"NaN": nan_values, "EmptySpaces": empty_spaces}) # New dataframe with the results
    print(f"There are {data.duplicated().sum()} duplicate rows. Also;")
    nulls = pd.DataFrame(data.isna().sum()*100/len(data), columns=['percentage'])
    nulls.sort_values('percentage', ascending = False)
    print(nulls)
    return exploration

#### import_data

In [3]:
data_df=pd.read_csv('D:\Ironhack\datos_proyecto\clean_data\out.csv')

In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18154 entries, 0 to 18153
Columns: 277 entries, cities to NAME
dtypes: float64(273), int64(1), object(3)
memory usage: 38.4+ MB


In [5]:
#Sort values to be able to process the function
#This is needed to apply next function
data_df=data_df.sort_values(by=['cities','TIME_PERIOD'],ascending=[True,False])

##### Create a dataframe with one city and all the variables. 

Here I get per city the most updated value for each variable. I don´t use the time series I get the most updated value for each variable. 

In [6]:
columns = data_df.columns #column names
subsets = data_df['cities'].unique() # we need one row per city

# Create an empty DataFrame to store the results
result_df = pd.DataFrame(columns=columns)

# Loop through each city. All years. 
for subset in subsets:
    subset_data = data_df[data_df['cities'] == subset]
    subset_values = {}

    # Loop through each column
    for column in columns:
        column_values = subset_data[column]

        # Find the first non-null value. I have ordered the dataframe to have the newest the first
        first_non_null = column_values.dropna().iloc[0] if not column_values.isnull().all() else None

        # Store the first non-null value for the column
        subset_values[column] = first_non_null

    # Convert the subset_values dictionary to a DataFrame
    subset_df = pd.DataFrame(subset_values, index=[0])

    # Concatenate the subset DataFrame to the result DataFrame
    result_df = pd.concat([result_df, subset_df], ignore_index=True)



In [7]:
result_df.shape

(910, 277)

In [8]:
result_df.sample(10)

,cities,TIME_PERIOD,TE1001I,TE1001V,TE1026I,TE1026V,TE1027V,TE1028V,TE1039V,TE1040V,TE1041V,TE2025V,TE2028I,TE2028V,TE2031I,TE2031V,EN1002V,EN1003V,EN1004V,EN1005V,EN2002V,EN2003V,EN2005V,EN2025V,EN2026V,EN2027V,EN2029I,EN2032I,EN2033I,EN2035I,EN2036I,EN3003V,EN3010V,EN3011V,EN3012V,EN3013V,EN4008V,EN5200V,EN5201V,EN5202V,EN5203V,EN5204V,EN5205V,EN5206V,EN5207V,SA2004I,SA2004V,SA2007I,SA2007V,SA2010V,SA2013V,SA2016V,SA2017V,SA2018V,SA2019V,SA2020V,SA2021V,SA2029I,DE3001V,DE3002I,DE3002V,DE3004I,DE3005I,DE3005V,DE3008I,DE3008V,DE3011I,DE3011V,DE3016I,DE3017V,EC3039V,EC3040V,EC3064V,EC3065V,EC3066V,EC3067V,SA1001V,SA1004V,SA1005V,SA1007I,SA1007V,SA1008I,SA1008V,SA1011I,SA1011V,SA1012V,SA1013V,SA1018V,SA1022V,SA1025V,SA1029V,SA1049V,SA1050V,SA1051V,SA1052V,SA1053V,SA3005V,EC1001I,EC1002I,EC1002V,EC1003I,EC1003V,EC1010V,EC1011I,EC1011V,EC1012I,EC1012V,EC1020I,EC1145V,EC1146V,EC1147V,EC1174V,EC1175V,EC1176V,EC1177V,EC1178V,EC1179V,EC1180V,EC1181V,EC1182V,EC2008V,EC2009V,EC2020V,EC2022V,EC2032V,EC2033V,EC2034V,EC2035V,EC2036V,EC2037V,EC2038V,DE1001V,DE1002V,DE1003V,DE1025V,DE1026V,DE1027V,DE1028V,DE1029V,DE1030V,DE1040V,DE1041V,DE1042V,DE1046V,DE1047V,DE1048V,DE1049V,DE1050V,DE1051V,DE1055V,DE1056V,DE1057V,DE1058V,DE1059V,DE1060V,DE1061V,DE1062V,DE1063V,DE1064V,DE1065V,DE1066V,DE1074V,DE1075V,DE1076V,DE1077V,DE1078V,DE1079V,DE1101V,DE1102V,DE1103V,DE1104V,DE1105V,DE1106V,DE1107V,DE1108V,DE1109V,DE1110V,DE1111V,DE1112V,DE1113V,DE1114V,DE1115V,DE1116V,DE1117V,DE1118V,DE1119V,DE1120V,DE1121V,DE1122V,DE1123V,DE1124V,DE1125V,DE1126V,DE1127V,DE1128V,DE1129V,DE1130V,DE1131V,DE1132V,DE1133V,DE1134V,DE1135V,DE1136V,DE1137V,DE1138V,DE1139V,DE2001I,DE2001V,DE2002I,DE2002V,DE2003I,DE2003V,DE2007I,DE2008I,DE2008V,DE2009I,DE2009V,DE2010V,DE2011V,DE2012V,DE4001V,DE4002V,DE4003V,DE4004V,DE4005V,TT1003V,TT1006V,TT1007V,TT1008V,TT1010V,TT1012V,TT1019V,TT1020V,TT1057I,TT1057V,TT1060I,TT1060V,TT1064V,TT1065V,TT1079V,TT1080V,TT1081V,CR1003I,CR1003V,CR1005V,CR1007V,CR1008V,CR1010V,CR1015V,CR2001V,CR2009V,CR2010I,CR2010V,CR2011I,CR2011V,DE1003I,DE1025I,DE1028I,DE1040I,DE1046I,DE1049I,DE1055I,DE1057I,DE1058I,DE1059I,DE1060I,DE1061I,DE1064I,DE1073I,DE1073V,DE1076I,DE1084I,DE1085I,EC2021V,EC2039V,CODE,NAME
682,PT014C,2022,None,None,0.0,190.0,143.0,47.0,None,None,None,5538.0,29.0,4880.0,13.0,2933.0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,33190680.0,None,None,100.0,93.0,195638.0,None,None,None,None,None,None,None,None,4.0,12.0,7.0,2736.0,72.0,113.0,404.0,252.0,217.0,3500.0,1809.0,1691.0,9.0,153222.0,25.0,37771.0,2.0,3.0,5119.0,36.0,55694.0,19.0,28497.0,18.0,381328.0,None,None,None,None,None,None,184483.0,44704.0,11681.0,None,35068.0,None,108721.0,78.0,105214.0,1638.0,27249.0,934.0,37.0,153147.0,None,7.0,342970.0,133378.0,1957.0,1536.0,3.0,68.0,74.0,99888.0,63.0,94950.0,17736.0,8.0,7833.0,10.0,9903.0,9.0,21833.0,11295.0,10538.0,188883.0,91662.0,97221.0,171685.0,84110.0,87575.0,18879.0,9753.0,9126.0,1033.0,18260.0,116817.0,12492.0,39098.0,2697.0,712.0,2416.0,26192.0,8384.0,6245.0,385777.0,181561.0,204216.0,49173.0,23246.0,25927.0,38190.0,17067.0,21123.0,18266.0,9305.0,8961.0,21987.0,11211.0,10776.0,23895.0,11928.0,11967.0,28657.0,11518.0,17139.0,43495.0,20616.0,22879.0,55898.0,25685.0,30213.0,65906.0,31190.0,34716.0,18113.0,9129.0,8984.0,20457.0,10373.0,10084.0,23818.0,11711.0,12107.0,23924.0,11464.0,12460.0,24513.0,11660.0,12853.0,28272.0,13189.0,15083.0,31845.0,14851.0,16994.0,30459.0,14030.0,16429.0,27339.0,12661.0,14678.0,22922.0,10499.0,12423.0,19748.0,8781.0,10967.0,17708.0,7772.0,9936.0,14206.0,6219.0,7987.0,9590.0,3901.0,5689.0,8715.0,2877.0,5838.0,100.0,385606.0,1.0,4500.0,10.0,37975.0,11.0,81.0,312991.0,19.0,72615.0,10036.0,62579.0,42475.0,325081.0,22107.0,28055.0,813.0,793.0,69.01,0.68,0.29,12.86,15.31,69.69,23.76,None,None,None,0.16,6.0,31825.0,110110.0,None,73.75,None,5.0,2054.0,85017.0,819989.0,6.0,None,None,247778.0,2586.0,7.0,158394.0,1.0,89384.0,112.5,12.7,9.8,4.7,5.7,6.2,7.4,148.8,64.8,36.4,28.4,-0.1,17.0,11.2

<div class="alert alert-block alert-warning">
<b>Output:</b> Export checkpoint 1 to a csv
</div>

In [9]:
result_df.to_csv('D:\Ironhack\datos_proyecto\clean_data\cities_clean_1.csv', index=False)

In [10]:
result_df.shape

(910, 277)

In [11]:
#Export a dictionary with the percentage nulls to display it in Tableau
nulls = pd.DataFrame(result_df.isna().sum()*100/len(result_df), columns=['percentage'])
nulls.sort_values('percentage', ascending = False)
#Import dictionary and check which variables still have a lot of nulls
dictionary_df=pd.read_csv('D:\Ironhack\datos_proyecto\clean_data\dictionary.csv')
#dictionary_df=pd.merge(left=dictionary_df,right=df_headers,how='left',left_on=['Column Name'],right_on=['Column Name'])
dictionary_df=dictionary_df.reset_index(drop=True)
nan_df=pd.merge(left=nulls,right=dictionary_df,how='left',left_index=True,right_on=['Column Name'])
nan_df=nan_df.reset_index(drop=True)
nan_df.sort_values(by='percentage',ascending=False)

,percentage,Column Name,Dataframe,Measurement unit,label
44,98.461538,EN2029I,environment,NaN,Prop. of residents exposed to air traffic nois...
45,97.142857,EN2032I,environment,NaN,Prop. of residents exposed to rail traffic noi...
48,96.703297,EN2036I,environment,NaN,Prop. of residents exposed to rail traffic noi...
47,96.373626,EN2035I,environment,NaN,Prop. of residents exposed to road traffic noi...
46,96.043956,EN2033I,environment,NaN,Prop. of residents exposed to road traffic noi...
93,92.417582,EC3067V,living_conditions,NaN,Share of persons at risk of poverty or social ...
92,92.417582,EC3066V,living_conditions,NaN,Share of severely materially deprived persons -%
28,80.329670,TE1041V,education,NaN,Share of early leavers from education and trai...
27,80.219780,TE1040V,education,NaN,Share of early leavers from education and trai...
26,79.120879,TE1039V,education,NaN,Share of early leavers from education and trai...


<div class="alert alert-block alert-warning">
<b>Output:</b> Export dictionary for tableau
</div>

In [ ]:
nan_df.to_excel(r'C:\Users\luisj\Desktop\Ironhack\Archivos Ejercicios\09_Final project\Data\dictionary_tableau.xlsx', index=False, sheet_name='dictionary')

<div class="alert alert-block alert-warning">
<b>Output:</b> Export excel with all data for Tableau
</div>

In [158]:
dictionary_2 = dictionary_df.set_index('Column Name')['label'].to_dict()
df_tableau = result_df.rename(columns=dictionary_2)
#df_tableau.to_csv('D:\Ironhack\datos_proyecto\clean_data\df_tableau_variables.csv', encoding='utf-8',index=False)
df_tableau.to_excel(r'C:\Users\luisj\Desktop\Ironhack\Archivos Ejercicios\09_Final project\Tableau\Sources\df_tableau_variables.xlsx', index=False, sheet_name='df_tableau_variables')

In [159]:
df_tableau.head(5)

,NaN,NaN,Number of Children 0-4 in day care (publ.&priv) per 1000 children 0-4,Number of children 0-4 in day care or school,Share of students in higher education in the total population (per 1000 persons),"Students in higher education (ISCED level 5-8), total","Students in higher education (ISCED level 5-8), male","Students in higher education (ISCED level 5-8), female","Share of early leavers from education and training, total -%","Share of early leavers from education and training, male -%","Share of early leavers from education and training, female -%","Persons aged 25-64 with ISCED level 0, 1 or 2 as the highest level of education",Prop. of working age population qualified at level 3 or 4 ISCED,Persons aged 25-64 with ISCED level 3 or 4 as the highest level of education,Prop. of working age population qualified at level 5 or 6 ISCED,"Persons aged 25-64 with ISCED level 5, 6, 7 or 8 as the highest level of education",Total number of hours of sunshine per day,Average temperature of warmest month - degrees,Average temperature of coldest month - degrees,Rainfall - litre/m²,Number of days ozone O3 concentrations exceed 120 µg/m³,Number of hours nitrogen dioxide NO2 concentrations exceed 200 µg/m³,Number of days particulate matter PM10 concentrations exceed 50 µg/m³,Accumulated ozone concentration in excess 70 µg/m³,Annual average concentration of NO2 (µg/m³),Annual average concentration of PM10 (µg/m³),Prop. of residents exposed to air traffic noise >55 dB(A) at night time,Prop. of residents exposed to rail traffic noise >65 dB(A) at day time,Prop. of residents exposed to road traffic noise >65 dB(A) at day time,Prop. of residents exposed to road traffic noise >55 dB(A) at night time,Prop. of residents exposed to rail traffic noise >55 dB(A) at night time,Total use of water - m³,Price of a m³ of domestic water - Euro,Share of the urban waste water load (in population equivalents) treated according to the applicable standard -%,Share of population connected to potable drinking water system - %,Share of population connected to sewerage treatment system - %,"Municipal waste generated (domestic and commercial), total",Share of land (%): Continuous residential urban fabric,Share of land (%): Discontinuous residential urban fabric,"Share of land (%): Industrial, commercial, public, military and private units",Share of land (%): Transport infrastructure,Share of land (%): Other artificial areas,Share of land (%): Green urban areas and sports and leisure facilities,Share of land (%): Agricultural areas,Share of land (%): Natural areas,Infant mortality rate (per 1000 live births),Infant Mortality per year,Crude birth rate (per 1000 inhabitants),Number of live births per year,Number of adolescent births (from mothers aged 19 or less),Number of deaths per year under 65 due to diseases of the circulatory or respiratory systems,Total deaths under 65 per year,"Total deaths under 65 per year, male","Total deaths under 65 per year, female",Total deaths per year,"Total deaths per year, male","Total deaths per year, female",Crude death rate (per 1000 inhabitants),Private households (excluding institutional households),Proportion of households that are 1-person households,One person households,Average size of households,Prop. of households that are lone-parent households,Lone parent private households (with children aged 0 to under 18),Prop. households that are lone-pensioner households,Lone pensioner (above retirement age) households,Proportion of households with children aged 0-17,Households with children aged 0 to under 18,Lone parent households per 100 households with children aged 0-17,Population living in private households (excluding institutional households),Median disposable annual household income - EUR,Average disposable annual household income - EUR,Share of persons living in households with very low work intensity - %,Share of persons at risk of poverty after social transfers -%,Share of severely materially deprived persons -%,Share 

#### Delete some Columns

In [13]:
#See the nulls in percentage
nulls = pd.DataFrame(result_df.isna().sum(), columns=['NaN'])
nulls = nulls.sort_values('NaN', ascending = False)
nan_df=pd.merge(left=nulls,right=dictionary_df,how='left',left_index=True,right_on=['Column Name'])
nan_df=nan_df.reset_index(drop=True)
nan_df.sort_values(by='NaN',ascending=False)

,NaN,Column Name,Dataframe,Measurement unit,label
0,896,EN2029I,environment,NaN,Prop. of residents exposed to air traffic nois...
1,884,EN2032I,environment,NaN,Prop. of residents exposed to rail traffic noi...
2,880,EN2036I,environment,NaN,Prop. of residents exposed to rail traffic noi...
3,877,EN2035I,environment,NaN,Prop. of residents exposed to road traffic noi...
4,874,EN2033I,environment,NaN,Prop. of residents exposed to road traffic noi...
5,841,EC3067V,living_conditions,NaN,Share of persons at risk of poverty or social ...
6,841,EC3066V,living_conditions,NaN,Share of severely materially deprived persons -%
7,731,TE1041V,education,NaN,Share of early leavers from education and trai...
8,730,TE1040V,education,NaN,Share of early leavers from education and trai...
9,720,TE1039V,education,NaN,Share of early leavers from education and trai...


In [14]:
columns_to_drop=nan_df[nan_df['NaN']>250] # select the limit of Nan values to filter out columns with more Nans
columns_to_drop=columns_to_drop['Column Name'].tolist()
columns_to_drop
data_clean_2=result_df.drop(columns=columns_to_drop)
data_clean_2.shape

(910, 162)

In [15]:
nan_df[nan_df['NaN']>100]['Column Name']

0      EN2029I
1      EN2032I
2      EN2036I
3      EN2035I
4      EN2033I
5      EC3067V
6      EC3066V
7      TE1041V
8      TE1040V
9      TE1039V
10     SA1052V
11     SA1053V
12     DE4004V
13     DE4005V
14     DE4002V
15     DE4001V
16     DE4003V
17     EN3013V
18     TT1020V
19     EC3064V
20     EN3011V
21     EC3065V
22     CR2011V
23     CR2010V
24     SA1049V
25     EN3012V
26     EC3039V
27     SA1029V
28     EN1002V
29     EN5206V
30     EN2025V
31     EN5200V
32     EN5201V
33     EN5204V
34     EN5207V
35     EN5205V
36     EN5203V
37     EN5202V
38     TT1080V
39     EN1003V
40     EN1005V
41     EN1004V
42     EC3040V
43     EN2027V
44     DE1102V
45     DE1135V
46     DE1103V
47     DE1136V
48     DE1138V
49     DE1137V
50     DE1139V
51     DE1133V
52     DE1132V
53     DE1101V
54     DE1134V
55     DE1131V
56     DE1130V
57     DE1129V
58     DE1128V
59     DE1127V
60     DE1126V
61     DE1125V
62     DE1120V
63     DE1124V
64     DE1104V
65     DE1105V
66     DE1

There are too many columns the only columns without missing is cities and Time period. 

- With less than 50 missing we get 83 variables
- With less than 100 missing we get 119 variables
- With less than 150 missing we get 137 variables
- With less than 200 missing we get 154 variables
- With less than 250 missing we get 162 variables

I will select anyway the variables manually to get the ones that are more important. 
Also the model will have for now not so many variables. It will have variables/parameters to check between 8 and 10

In [16]:
data_clean_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Columns: 162 entries, cities to NAME
dtypes: object(162)
memory usage: 1.1+ MB


In [17]:
data_clean_2.columns

Index(['cities', 'TIME_PERIOD', 'TE1026I', 'TE1026V', 'TE1027V', 'TE1028V',
       'TE2025V', 'TE2028I', 'TE2028V', 'TE2031I',
       ...
       'DE1061I', 'DE1064I', 'DE1073I', 'DE1073V', 'DE1076I', 'DE1084I',
       'DE1085I', 'EC2021V', 'CODE', 'NAME'],
      dtype='object', length=162)

In [18]:
nulls = pd.DataFrame(result_df.isna().sum()*100/len(result_df), columns=['percentage'])
nulls.sort_values('percentage', ascending = False)

,percentage
EN2029I,98.461538
EN2032I,97.142857
EN2036I,96.703297
EN2035I,96.373626
EN2033I,96.043956
EC3067V,92.417582
EC3066V,92.417582
TE1041V,80.329670
TE1040V,80.219780
TE1039V,79.120879


<div class="alert alert-block alert-warning">
<b>Output:</b> Export checkpoint 2 to a csv
</div>

In [19]:
data_clean_2.to_csv('D:\Ironhack\datos_proyecto\clean_data\cities_clean_2.csv', index=False)

Drop Turkey and Iceland and input EU Average in the rest of the countries if nulls for the selected variables. 
After that if a city has a null in a variable we will input the avg for that country. this should be the case for the indexes (variables with I at the end) and not for the  variables with V at the end (for example population) 

In [127]:
#Remove Turkey Iceland and three cities from France but outside Europe
result_df=result_df[~result_df['cities'].str.contains('TR')]
result_df=result_df[~result_df['cities'].str.contains('IS')]
#Removing french cities outside Europe
result_df=result_df[result_df['cities']!="FR046C"]
result_df=result_df[result_df['cities']!="FR072C"]
result_df=result_df[result_df['cities']!="FR062C"]

In [128]:
result_df.shape

(879, 277)

In [129]:
nulls = pd.DataFrame(result_df.isna().sum()*100/len(result_df), columns=['percentage'])
nulls.sort_values('percentage', ascending = False)

,percentage
EN2029I,98.407281
EN2032I,97.042093
EN2036I,96.587031
EN2035I,96.245734
EN2033I,95.904437
EC3067V,92.150171
EC3066V,92.150171
TE1041V,79.635950
TE1040V,79.522184
TE1039V,78.384528


In [130]:
#See the nulls in percentage
nan_df=pd.merge(left=nulls,right=dictionary_df,how='left',left_index=True,right_on=['Column Name'])
nan_df=nan_df.reset_index(drop=True)
nan_df.sort_values(by='percentage',ascending=False)

,percentage,Column Name,Dataframe,Measurement unit,label
44,98.407281,EN2029I,environment,NaN,Prop. of residents exposed to air traffic nois...
45,97.042093,EN2032I,environment,NaN,Prop. of residents exposed to rail traffic noi...
48,96.587031,EN2036I,environment,NaN,Prop. of residents exposed to rail traffic noi...
47,96.245734,EN2035I,environment,NaN,Prop. of residents exposed to road traffic noi...
46,95.904437,EN2033I,environment,NaN,Prop. of residents exposed to road traffic noi...
92,92.150171,EC3066V,living_conditions,NaN,Share of severely materially deprived persons -%
93,92.150171,EC3067V,living_conditions,NaN,Share of persons at risk of poverty or social ...
28,79.635950,TE1041V,education,NaN,Share of early leavers from education and trai...
27,79.522184,TE1040V,education,NaN,Share of early leavers from education and trai...
26,78.384528,TE1039V,education,NaN,Share of early leavers from education and trai...


i change the approach and I select variables I think are interesting for the model manually. The application shouldn´t have so many variables to manipulate by the user. If not is not fun for the user. 

In [131]:
#List of variables used
#features=['cities','TE2031I','TE1026I','DE1003I','DE1073V','DE3004I','EC1020I','DE3002I','TT1057I','TT1060I','DE2001I','CR2010I','SA2007I']
features=['cities','DE1001V','TE2031I','TE1026I','DE1003I','DE1073V','DE3004I','EC1020I','DE3002I','TT1057I','TT1060I','DE2001I','SA2007I']
#features=['DE1001V','DE1002V','EN1002V','EN1003V','EN1004V','EN1005V','SA1053V','EC1020I','EC1011I','EC1012I','TT1019V']
selected_df=result_df[features]
selected_df['cities_name']=result_df['NAME']

In [132]:
selected_df[selected_df['cities_name']=='Mamoudzou']

,cities,DE1001V,TE2031I,TE1026I,DE1003I,DE1073V,DE3004I,EC1020I,DE3002I,TT1057I,TT1060I,DE2001I,SA2007I,cities_name


In [133]:
nulls = pd.DataFrame(selected_df.isna().sum()*100/len(selected_df), columns=['percentage'])
nulls.sort_values('percentage', ascending = False)

,percentage
EC1020I,11.149033
TE1026I,10.466439
TT1057I,7.963595
TE2031I,6.939704
DE3004I,6.825939
DE3002I,5.005688
DE2001I,2.957907
TT1060I,2.844141
DE1073V,2.275313
DE1003I,0.341297


In [134]:
selected_df.sample(10)

,cities,DE1001V,TE2031I,TE1026I,DE1003I,DE1073V,DE3004I,EC1020I,DE3002I,TT1057I,TT1060I,DE2001I,SA2007I,cities_name
582,NL036C,81683.0,34.0,32.0,105.1,44.0,2.0,4.0,35.0,431.86,0.12,95.0,9.0,Purmerend
483,IT041C,84452.0,32.0,34.0,109.1,48.0,2.0,7.0,36.0,611.64,0.71,86.0,6.0,Treviso
638,PL039C,72050.0,36.0,5.0,109.8,42.7,None,None,None,1136.21,0.14,100.0,9.0,Ostrów Wielkopolski
808,UK034C,128781.0,27.0,None,103.5,41.4,2.0,5.0,30.0,492.79,0.08,92.0,12.0,Nuneaton and Bedworth
817,UK043C,99394.0,9.0,None,102.2,45.7,2.0,6.0,29.0,451.13,0.2,91.0,11.0,Great Yarmouth
144,DE070C,60388.0,30.0,69.0,103.7,45.0,2.0,4.0,45.0,37.62,0.17,86.0,10.0,Offenburg
113,DE039C,113388.0,35.0,142.0,104.2,42.5,2.0,5.0,51.0,452.85,0.26,86.0,10.0,Koblenz
908,UK134C,177310.0,42.0,93.0,107.2,43.1,2.0,5.0,35.0,434.7,0.23,95.0,10.0,Renfrewshire (Paisley)
552,NL006C,157462.0,56.0,41.0,102.4,39.0,2.0,4.0,37.0,402.26,0.13,96.0,12.0,Amersfoort
820,UK046C,99618.0,29.0,None,101.9,42.6,2.0,4.0,28.0,489.92,0.2,97.0,11.0,Cannock Chase


In [135]:
selected_df.shape

(879, 14)

In [136]:
selected_clean_df=selected_df[selected_df['cities'].str.len()!=2]

In [137]:
selected_clean_df.sample(1)

,cities,DE1001V,TE2031I,TE1026I,DE1003I,DE1073V,DE3004I,EC1020I,DE3002I,TT1057I,TT1060I,DE2001I,SA2007I,cities_name
110,DE036C,246601.0,40.0,147.0,105.4,39.8,2.0,5.0,56.0,385.5,0.2,88.0,10.0,Kiel


In [138]:
#Function to replace nan with mean in each column
def replace_nan_with_mean(df):
    # Calculate the mean of each column
    column_means = df.mean()

    # Replace NaN values with column means
    df_filled = df.fillna(column_means)

    return df_filled

#Function to replace nans with values for each country values.

#We have rows for all countries. So it make sense to replace the nans for cities with the value for each country and not the mean of all

In [139]:
countries_df = selected_df[selected_df['cities'].str.len()==2]

In [140]:
countries_df['country_cd']=countries_df['cities']
countries_df

,cities,DE1001V,TE2031I,TE1026I,DE1003I,DE1073V,DE3004I,EC1020I,DE3002I,TT1057I,TT1060I,DE2001I,SA2007I,cities_name,country_cd
0,AT,8507786.0,19.0,50.0,104.7,42.5,2.0,7.0,37.0,521.82,0.51,87.0,10.0,Österreich,AT
7,BE,11554767.0,35.0,34.0,102.7,41.0,2.0,5.0,35.0,513.03,0.45,87.0,10.0,Belgique/België,BE
23,BG,6951482.0,26.0,33.0,106.3,42.7,2.0,5.0,33.0,365.68,0.67,98.0,8.0,Bulgaria,BG
42,CH,8327126.0,37.0,39.0,102.0,42.0,2.0,5.0,35.0,543.29,0.26,75.0,11.0,Schweiz/Suisse,CH
53,CY,848300.0,33.0,29.0,105.5,None,3.0,11.0,21.0,501.27,1.16,79.0,11.0,Kypros/Kibris,CY
55,CZ,10610055.0,17.0,28.0,103.3,40.1,2.0,10.0,33.0,436.24,0.47,94.0,11.0,Česká republika,CZ
74,DE,83155031.0,29.0,34.0,102.7,46.0,2.0,3.0,40.0,491.61,0.4,88.0,10.0,Deutschland,DE
202,DK,5602628.0,33.0,44.0,101.6,40.7,2.0,4.0,39.0,350.75,0.68,94.0,12.0,Danmark,DK
207,EE,1319133.0,40.0,35.0,112.4,41.0,2.0,5.0,40.0,418.66,0.51,85.0,11.0,Eesti,EE
211,EL,10816286.0,26.0,32.0,104.0,41.0,3.0,19.0,26.0,479.21,1.05,92.0,10.0,Hellas/Greece,EL


In [141]:
countries_df.shape

(29, 15)

In [142]:
#Clean the nulls with the means of all dataset
#This was first Idea now cleaning the data with the value of each country for the cities and if there is no value the 
#mean for all dataset
#dataoutput=replace_nan_with_mean(selected_clean_df)

In [143]:
#dataoutput.sample(10)

In [144]:
#nulls = pd.DataFrame(dataoutput.isna().sum()*100/len(dataoutput), columns=['percentage'])
#nulls.sort_values('percentage', ascending = False)

In [145]:
#dataoutput.describe()

In [146]:
selected_clean_df.columns

Index(['cities', 'DE1001V', 'TE2031I', 'TE1026I', 'DE1003I', 'DE1073V',
       'DE3004I', 'EC1020I', 'DE3002I', 'TT1057I', 'TT1060I', 'DE2001I',
       'SA2007I', 'cities_name'],
      dtype='object')

In [147]:
#Clean the nulls with the value for the country
#Create country column
selected_clean_df['country_cd'] = selected_clean_df['cities'].str[:2]
df_merged = selected_clean_df.merge(countries_df, on='country_cd', how='left')
df_merged=df_merged.drop(columns=['country_cd'])
print(df_merged.columns)
columns=selected_clean_df.columns
columns = [item for item in columns if item != "country_cd"]
for feature in columns:
    df_merged[feature + '_x'] = df_merged[feature + '_x'].combine_first(df_merged[feature + '_y'])




Index(['cities_x', 'DE1001V_x', 'TE2031I_x', 'TE1026I_x', 'DE1003I_x',
       'DE1073V_x', 'DE3004I_x', 'EC1020I_x', 'DE3002I_x', 'TT1057I_x',
       'TT1060I_x', 'DE2001I_x', 'SA2007I_x', 'cities_name_x', 'cities_y',
       'DE1001V_y', 'TE2031I_y', 'TE1026I_y', 'DE1003I_y', 'DE1073V_y',
       'DE3004I_y', 'EC1020I_y', 'DE3002I_y', 'TT1057I_y', 'TT1060I_y',
       'DE2001I_y', 'SA2007I_y', 'cities_name_y'],
      dtype='object')


In [148]:
#Clean columns duplicated
columns_to_drop = [col for col in df_merged.columns if '_y' in col]
columns_to_drop
df_nan_replaced=df_merged.drop(columns=columns_to_drop)

In [149]:
columns_to_rename = [col for col in df_nan_replaced.columns if col.endswith('_x')]
new_column_names = [col.replace('_x', '') for col in columns_to_rename]
df_nan_replaced.rename(columns=dict(zip(columns_to_rename, new_column_names)), inplace=True)


In [150]:
df_nan_replaced.sample(5)

,cities,DE1001V,TE2031I,TE1026I,DE1003I,DE1073V,DE3004I,EC1020I,DE3002I,TT1057I,TT1060I,DE2001I,SA2007I,cities_name
620,PL048C,63952.0,26.0,34.0,108.6,41.5,3.0,6.0,23.0,571.41,0.16,100.0,10.0,Leszno
508,IT086C,55035.0,15.0,0.0,102.0,45.0,3.0,15.0,21.0,616.14,0.0,97.0,7.0,Trani
656,PT016C,137305.0,10.0,0.0,109.8,43.0,2.0,8.0,24.0,None,0.44,100.0,7.0,Vila Franca de Xira
64,CZ015C,71200.0,16.0,4.0,105.5,41.9,2.0,16.0,35.0,359.22,0.0,96.0,9.0,Havířov
45,CH007C,115129.0,50.0,120.0,102.3,39.2,2.0,6.0,39.0,397.63,0.26,75.0,12.0,Winterthur


In [151]:
#There is still nulls clean them with the mean for all dataset
explore_data(df_nan_replaced)

There are 0 duplicate rows. Also;
             percentage
cities         0.000000
DE1001V        0.000000
TE2031I        0.235294
TE1026I        0.000000
DE1003I        0.000000
DE1073V        0.941176
DE3004I        0.235294
EC1020I        0.000000
DE3002I        0.000000
TT1057I        1.882353
TT1060I        0.823529
DE2001I        0.000000
SA2007I        0.000000
cities_name    0.000000


,NaN,EmptySpaces
cities,0,0
DE1001V,0,0
TE2031I,2,0
TE1026I,0,0
DE1003I,0,0
DE1073V,8,0
DE3004I,2,0
EC1020I,0,0
DE3002I,0,0
TT1057I,16,0


there are still null that I will replace with the mean of all

In [152]:
df_nan_replaced=replace_nan_with_mean(df_nan_replaced)

In [153]:
explore_data(df_nan_replaced)

There are 0 duplicate rows. Also;
             percentage
cities              0.0
DE1001V             0.0
TE2031I             0.0
TE1026I             0.0
DE1003I             0.0
DE1073V             0.0
DE3004I             0.0
EC1020I             0.0
DE3002I             0.0
TT1057I             0.0
TT1060I             0.0
DE2001I             0.0
SA2007I             0.0
cities_name         0.0


,NaN,EmptySpaces
cities,0,0
DE1001V,0,0
TE2031I,0,0
TE1026I,0,0
DE1003I,0,0
DE1073V,0,0
DE3004I,0,0
EC1020I,0,0
DE3002I,0,0
TT1057I,0,0


<div class="alert alert-block alert-warning">
<b>Output:</b> Export data for model to a csv
</div>

In [154]:
#Put the names to columns
dictionary_2 = dictionary_df.set_index('Column Name')['label'].to_dict()
df_model_output = df_nan_replaced.rename(columns=dictionary_2)
df_model_output.sample(5)

,NaN,"Population on the 1st of January, total",Prop. of working age population qualified at level 5 or 6 ISCED,Share of students in higher education in the total population (per 1000 persons),Women per 100 men,Median population age,Average size of households,Unemployment rate,Proportion of households that are 1-person households,Number of registered cars per 1000 population,People killed in road accidents per 10000 pop.,Nationals as a proportion of population,Crude birth rate (per 1000 inhabitants),cities_name
577,PL005C,536438.0,37.0,139.0,114.4,41.6,2.0,6.0,33.0,725.080000,0.48,100.0,12.0,Poznań
217,ES007C,459403.0,39.0,115.0,104.2,41.4,3.0,16.0,24.0,487.590000,0.17,87.0,9.0,Murcia
307,ES097C,63563.0,32.0,12.0,101.7,42.9,3.0,16.0,26.0,489.590000,0.16,89.0,8.0,Igualada
554,NL033C,177359.0,67.0,112.0,106.1,36.0,2.0,6.0,53.0,375.890000,0.06,94.0,10.0,Nijmegen
655,PT015C,156348.0,6.0,0.0,106.5,46.0,3.0,7.0,17.0,477.466571,0.64,100.0,7.0,Guimarães


In [155]:
df_model_output.to_csv(r'C:\Users\luisj\Desktop\Ironhack\Archivos Ejercicios\09_Final project\Data\cities_clean_model.csv', index=False)

<div class="alert alert-block alert-warning">
<b>Output:</b> Export the data for the data model to excel for Tableau
</div>

In [156]:
#Data ouput for Tableau
dataoutput_tableau = df_model_output.rename(columns=dictionary_2)
dataoutput_tableau.to_excel(r'C:\Users\luisj\Desktop\Ironhack\Archivos Ejercicios\09_Final project\Data\model_tableau.xlsx', index=False, sheet_name='dictionary')